In [1]:
import numpy as np
import math
import random as rn
import pandas as pd
from library import Map, GradientDescent
import time
from sklearn.linear_model import SGDClassifier
from random import randrange
from sklearn.linear_model import LinearRegression

In [5]:
'''Downloading dataset'''
dataset = pd.read_csv("/Users/alessandrococcia/Downloads/ObesityDataSet.csv")

'''Dataset sampling'''
dataset.sample(frac=1, ignore_index=True) #shuffle sample in the training set

'''mapping delle stringhe'''

dataset = pd.get_dummies(dataset, drop_first=True, dtype=float)

'''normalization'''
#dataset = (dataset-dataset.min())/(dataset.max()-dataset.min())
dataset =(dataset-dataset.mean())/dataset.std()

'''Inserimento colonna di bias'''
dataset.insert(0, "Bias", np.ones(len(dataset)), True) #Bias row

TRAIN_TEST_SPLIT_PERCENTAGE = 0.9
dataset_training = dataset[:int(len(dataset) * TRAIN_TEST_SPLIT_PERCENTAGE)]
dataset_test = dataset[int(len(dataset) * TRAIN_TEST_SPLIT_PERCENTAGE):]


In [3]:
class Map:
    def __init__(self):
        self.count = 0
        self.count_dataset = {}
        self.d = {}
        self.dataset_dictionary = {}
        self.matrice = None
        self.dataset = None
        len = None
    
    def mappingElement(self, string):
        if type(string) is str:
            if self.d.get(string) is not None:
                return self.d.get(string)
            else:
                self.count = self.count+1
                self.d[string] = self.count
                return self.count
        return float(string)
    
    def mappingMatrix(self, matrice):
        self.matrice = None
        self.matrice = np.array(matrice)
        len = self.matrice.shape
        for g in range(len[0]):
            for j in range(len[1]):
                self.matrice[g][j] = self.mappingElement(self.matrice[g][j])
        return np.float64(self.matrice.copy())
    
    def mappingElementDataset(self, value, colonna):
        if type(value) is str:
            if self.dataset_dictionary[colonna].get(value) is not None:
                return self.dataset_dictionary[colonna].get(value)
            else:
                if self.count_dataset.get(colonna) is None:
                    self.count_dataset[colonna] = 0
                    self.dataset_dictionary[colonna][value] = self.count_dataset[colonna]
                    return self.count_dataset[colonna]
                self.count_dataset[colonna] += 1
                self.dataset_dictionary[colonna][value] = self.count_dataset[colonna]
                return self.count_dataset[colonna]
        return value
    
    def mappingDataset(self, dataset):
        self.dataset = dataset
        colonne = dataset.columns
        length = len(dataset)
        for column in colonne:
            self.dataset_dictionary[column] = {}
            for index in range(length):
                dataset.at[index, column] = self.mappingElementDataset(dataset.at[index, column], column)
        return self.dataset

    def MSE(self, a, b):
        return np.mean((np.square(a - b)))
    def RMSE(self, a, b):
        return math.sqrt(np.mean(np.square(a - b)))
    def MAE(self, a, b):
        return np.mean(abs(a-b))
    

class DecisionTree:
    def __init__(self, outputstring, positive, negative): 
        self.outputstring = outputstring
        self.positive = positive
        self.negative = negative
        self.nodes = []
        self.label = None
        self.root = False

    def LearnDecisionTree(self, examples, attributes, parent_examples, column_values):
        self.root = True
        self.LearnDecisionTreeAux_(examples, attributes, parent_examples, column_values)

    def LearnDecisionTreeAux_(self, examples, attributes, parent_examples, column_values):
        same_classification = self.SameClassification(examples)    
        
        if len(examples.loc[:, examples.columns != self.outputstring]) == 0:
            return self.PluralityValue(parent_examples)
        elif same_classification is not False:
            return same_classification
        elif len(attributes) == 0: 
            return self.PluralityValue(examples)
        else:
            '''Seleziono l'attributo migliore'''
            bestattribute = self.Importance(attributes, examples)
            self.label = bestattribute #Seleziono l'attributo migliore
            
            for value in self.Values1(bestattribute, column_values):
                remainingexamples = self.Examples(bestattribute, examples, value)
                tree = DecisionTree(self.outputstring, self.positive, self.negative)
                attributes_left = self.PopListValue(attributes.copy(), bestattribute)                
                subtree = tree.LearnDecisionTreeAux_(remainingexamples.loc[:, remainingexamples.columns != bestattribute], attributes_left, examples, column_values)
                self.nodes.append((value, subtree))
        return self
        
    def Importance(self, attributes, examples): 
        max = -1
        ret = None
        for a in attributes:
            loc = self.Gain(examples, a)
            if loc > max:
                max = loc
                ret = a
        return ret
        
    def B(self, q):
        if q == 1 or q == 0:
            return 0
        return -(q*math.log2(q)+(1-q)*math.log2(1-q))
    
    def Remainder(self, examples, attribute, p, n):
        sum = 0
        for v in self.Values2(attribute, examples):
            if type(v) is not str and math.isnan(float(v)):
                pk = len(examples.loc[(examples[self.outputstring] == self.positive) & (examples[attribute].isnull())])
                nk = len(examples.loc[(examples[self.outputstring] == self.negative) & (examples[attribute].isnull())])
            else:
                pk = len(examples.loc[(examples[self.outputstring] == self.positive) & (examples[attribute] == v)])
                nk = len(examples.loc[(examples[self.outputstring] == self.negative) & ((examples[attribute] == v))])
            
            partial = (pk+nk)*self.B(pk/(pk+nk))
            sum += partial
        return (1/(p+n))*sum
    
    def Gain(self, examples, attribute):
        p = len(examples.loc[examples[self.outputstring] == self.positive])
        n = len(examples.loc[examples[self.outputstring] == self.negative])
        b = self.B(p/(p+n))
        r = self.Remainder(examples, attribute, p, n)
        return (b-r)
        
    def PluralityValue(self, parent_examples): 
        '''Selects the most common ouput value among a set of examples, breaking ties randomly'''
        value, max = [], 0
        d = self.CreateDictionary(parent_examples)
        for key in d.keys():
            if d.get(key) > max:
                max = d.get(key)
                value = []
                value.append(key)
            elif d.get(key) == max:
                '''se ci sono più massimi li metto tutti in una lista dalla quale ne
                    selezionerò uno randomicamente
                '''
                value.append(key)
        return rn.choice(value)
        
    def SameClassification(self, examples): 
        if (len(examples) == 0):
            return False
        d = self.CreateDictionary(examples)
        if (len(d.keys()) == 1):
            return list(d.keys())[0]
        return False
    def CreateDictionary(self, examples):
        examples = examples[self.outputstring].tolist()
        d = {}
        for i in range(len(examples)):
            if d.get(examples[i]) is None:
                d[examples[i]] = 1
            else:
                d[examples[i]] += 1
        return d

    def Values1(self, attribute, dictionary): 
        return list(dictionary.get(attribute))
    
    def Values2(self, attribute, examples):
        return examples[attribute].unique()

    def Examples(self, attribute, examples, value):
        exs = examples.loc[examples[attribute] == value]
        return exs
    def PopListValue(self, lista, value):
        if value not in lista:
            return None
        lista.remove(value)
        return lista 
    def PrintDecisionTree(self, count):
        print(" "*count, self.label)
        for elem in self.nodes:
            if type(elem[1]) is not DecisionTree:
                print("    "*(count+1), elem[0], " --> ", elem[1])
            else:
                elem[1].PrintDecisionTree(count+1)

    def Prediction(self, input):
        for i in range(len(self.nodes)):
            x = input[self.label].values[0]
            if type(self.nodes[i][0]) is not str and math.isnan(float(self.nodes[i][0])):
                if type(x) is not str and math.isnan(float(x)):
                    if type(self.nodes[i][1]) is DecisionTree:
                        return self.nodes[i][1].Prediction(input)
                    return self.nodes[i][1]
            elif self.nodes[i][0] == x:
                if type(self.nodes[i][1]) is DecisionTree:
                    return self.nodes[i][1].Prediction(input)
                return self.nodes[i][1]
        return None

class GradientDescent:
    def __init__(self, learningrate, epochs, parameters_length, output_string, batch_size): 
        self.learningrate = learningrate
        self.epochs = epochs
        self.parameters_length = parameters_length
        self.parameters = None
        self.output_string = output_string
        self.len_totale = None
        self.batch_size = batch_size

    def SGD(self, training_set):
        
        '''Inizializzo i pesi ad 1'''
        self.parameters = np.ones(self.parameters_length)
        '''Su input e output totale calcolerò ad ogni n-esima epoca l'errore'''
        input_totale = training_set.loc[:len(training_set), training_set.columns != self.output_string]
        output_totale = training_set.loc[:len(training_set), training_set.columns == self.output_string]
        self.len_totale = len(training_set)

        '''ciclo for per le epoche'''
        '''Batch == input totale: non devo randomizzare'''
        if self.batch_size == self.len_totale:
            input = input_totale
            output = output_totale
            for i in range(self.epochs):
                gradient = self.gradient(input, output)
                self.SGD_aux(gradient)
                if (not (i%10000)):
                    print("Epoca: \t", i)
                    print("MSE: ", self.MSE(self.prediction(input_totale), output_totale))
        
        else:
            '''uso delle minibatch che devono avere i sample randomizzati ad ogni iterazione'''
            for i in range(self.epochs):
                batch = training_set.sample(n=self.batch_size)
                input = batch.loc[:, training_set.columns != self.output_string]
                output = batch.loc[:, training_set.columns == self.output_string]
                gradient = self.gradient(input, output)
                self.SGD_aux(gradient)
                
                if (not (i%10000)):
                    print("Epoca: \t", i)
                    print("MSE: ", self.MSE(self.prediction(input_totale), output_totale))
                
        return self.parameters

    def SGD_aux(self, gradient):
        self.parameters -= self.learningrate*gradient

    def gradient(self, input, output): 
        output = np.array(output).T
        prediction = self.prediction(input)
        error = prediction - output
        return np.matmul(error, input)

    def prediction(self, input):
        return np.dot(input, self.parameters)

    def MSE(self, prediction, output):
        output = np.array(output)
        return np.mean((output - prediction)**2)
    def RMSE(self, prediction, output):
        return math.sqrt(np.mean((output - prediction)**2))


In [15]:
gd = GradientDescent(1e-3, 70000, dataset_training.shape[1]-1, 'Weight', 1)

start = time.time()
pesi = gd.SGD(dataset_training)
end = time.time()
print(end-start, pesi)

Epoca: 	 0
MSE:  24.64514178309623
Epoca: 	 10000
MSE:  1.7846712462168153
Epoca: 	 20000
MSE:  1.7210601241321803
Epoca: 	 30000
MSE:  1.707410737311722
Epoca: 	 40000
MSE:  1.7116694930861778
Epoca: 	 50000
MSE:  1.6984440438089992
Epoca: 	 60000
MSE:  1.6914073866750585
58.27085518836975 [ 8.53110778e-03 -1.21187547e-02  3.17042161e-01  6.46144282e-03
 -1.38957745e-02  4.03215871e-03  3.69772533e-04  4.59043057e-03
  9.70832699e-03 -1.54346309e-02 -5.89679692e-03  2.74287995e-02
 -6.69868473e-03 -8.34284738e-04  3.76998196e-03  3.88452925e-03
  3.87745863e-01  1.05098584e+00  9.99978617e-01 -1.24886622e-02
 -2.67415894e-03 -1.42466944e-02 -2.11525100e-03  1.83733661e-01
  6.20307119e-01  7.81983556e-01  1.00392616e+00  3.32767578e-01
  4.18182923e-01]


In [16]:
X_test = np.array(dataset_test.loc[:, dataset_test.columns != 'Weight'])
Y_effettivo = np.array(dataset_test['Weight'])
Y_predetto = np.array(np.matmul(X_test, pesi))



map = Map()
print("MSE: ", map.MSE(Y_effettivo, Y_predetto))
print("RMSE: ", map.RMSE(Y_effettivo, Y_predetto))
print("MAE: ", map.MAE(Y_effettivo, Y_predetto))

'''sklearn'''

X = np.array(dataset_training.loc[:, dataset_training.columns != 'Weight'])
Y = np.array(dataset_training.loc[:, dataset_training.columns == 'Weight'])

reg = LinearRegression().fit(X, Y)
scikit_predict = np.array(reg.predict(X_test))

print("MSE_sklearn: ", map.MSE(Y_effettivo, scikit_predict))
print("RMSE_sklearn: ", map.RMSE(Y_effettivo, scikit_predict))
print("MAE_sklearn: ", map.MAE(Y_effettivo, scikit_predict))

from sklearn.linear_model import SGDRegressor

# X_test e Y_effettivo sono i dati di test

# Definisci il regressore SGD
sgd_regressor = SGDRegressor(max_iter=100000000, alpha=0.001, random_state=42)

# Addestra il modello
sgd_regressor.fit(X, Y)

# Effettua previsioni
Y_predetto_sgd = np.array(sgd_regressor.predict(X_test))

# Calcola le metriche di valutazione

# Stampare le metriche di valutazione
print("MSE_sklearn SGD: ", map.MSE(Y_effettivo, Y_predetto_sgd))
print("RMSE_sklearn SGD: ", map.RMSE(Y_effettivo, Y_predetto_sgd))
print("MAE_sklearn SGD: ", map.MAE(Y_effettivo, Y_predetto_sgd))


MSE:  0.13812554467754154
RMSE:  0.3716524514617676
MAE:  0.3249197904591904
MSE_sklearn:  0.377664398692556
RMSE_sklearn:  0.6145440575683374
MAE_sklearn:  0.5249601168214079
MSE_sklearn SGD:  0.14643371987563464
RMSE_sklearn SGD:  0.38266659101054884
MAE_sklearn SGD:  0.3353679576421768


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/utils/validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
